In [7]:
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.preprocessing import normalize
import plotly.express as px

In [8]:
BASE = Path("input")
OUT = Path("output")
OUT.mkdir(parents=True, exist_ok=True)

In [9]:
# Загружаем CSV-файл
df = pd.read_csv(BASE / "Outbreaks.csv")

# Приводим названия колонок к нижнему регистру
df = df.rename(columns={"Country":"country","Year":"year","Disease":"disease","iso3":"iso3"})

# Удаляем строки без iso3 или disease
df = df.dropna(subset=["iso3","disease"])

# Строим сводную таблицу: сколько раз каждая болезнь была зафиксирована в каждой стране
pivot = (
    df.groupby(["iso3","disease"]).size().rename("cnt").reset_index()
      .pivot_table(index="iso3", columns="disease", values="cnt", fill_value=0)
)

# Нормализуем по L2-норме по строкам (странам)
pivot_norm = pd.DataFrame(normalize(pivot, norm="l2"), index=pivot.index, columns=pivot.columns)

# Количество кластеров
k = 5

# Обучаем KMeans
km = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = km.fit_predict(pivot_norm.values)

# Вычисляем метрики качества кластеризации
sil = float(silhouette_score(pivot_norm.values, labels))
ch  = float(calinski_harabasz_score(pivot_norm.values, labels))

# Сохраняем метки кластеров в файл
clusters = pd.DataFrame({"iso3": pivot_norm.index, "cluster": labels})
clusters.to_csv(OUT / "country_profiles_clusters.csv", index=False)

# Рисуем хлороплет-карту с кластерами
fig = px.choropleth(
    clusters, locations="iso3", color="cluster",
    locationmode="ISO-3", color_continuous_scale="Viridis",
    title=f"Эпидемиологические профили стран (кластеры k={k})"
)
fig.update_layout(margin=dict(l=0,r=0,t=50,b=0))

# Сохраняем карту в HTML
map_path = OUT / "country_profiles_map.html"
fig.write_html(map_path, include_plotlyjs="cdn")

# Сохраняем метрики в текстовый файл
with open(OUT / "country_profiles_metrics.txt","w") as f:
    f.write(f"k={k}\nSilhouette={sil:.3f}\nCalinski-Harabasz={ch:.0f}\n")